In [ ]:
import os
import json
import random
import torch
from dataclasses import dataclass, asdict
from transformers import AutoModel, AutoTokenizer
import pandas as pd
import time
import gc

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 如果GPU设备可以，则用GPU；否则用CPU设备

In [ ]:
# 获取数据
parquet_file_path = "train-00040-of-00041.parquet"
df = pd.read_parquet(parquet_file_path)
df.head()

In [ ]:
# 定义数据类
@dataclass
class NeuronId:
    layer_index: int  # 神经元所在的层的索引
    neuron_index: int  # 神经元在层中的索引

@dataclass
class ActivationRecord:
    tokens: list  # 输入 token 序列
    activations: list  # 对应的激活值序列

@dataclass
class NeuronRecord:
    dataclass_name: str  # 数据类名称
    neuron_id: NeuronId  # 神经元的 ID
    random_sample: list  # 随机样本序列

In [ ]:
# 创建 GLM 激活值文件夹
glm_activation_folder = "GLM激活值"
os.makedirs(glm_activation_folder, exist_ok=True)

In [ ]:
# 创建 GLM 激活值文件夹
glm_activation_folder = "GLM激活值"
os.makedirs(glm_activation_folder, exist_ok=True)

In [ ]:
global neuron_records,neuron_records_all
# 创建 hook 函数来记录激活值
def hook(inputs, output):
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().tolist())  # 将输入编码的标记转换为文本标记
    tokens = [token for token in tokens if token.strip()]  # 过滤掉特殊标记并转换为列表
    activations = []
    for i in output[0]:
        i = i.squeeze().tolist()
        activations.append(i[neuron_index])
    activation_record = ActivationRecord(tokens=tokens, activations=activations)  # 创建 ActivationRecord 对象
    neuron_id = NeuronId(layer_index=layer_index, neuron_index=neuron_index)  # 创建 NeuronId 对象
    neuron_record = NeuronRecord(dataclass_name="NeuronRecord", neuron_id=neuron_id, random_sample=[activation_record])  # 创建 NeuronRecord 对象
    neuron_records.append(neuron_record)  # 添加到记录列表中

In [ ]:
# 获取28层的4096个神经元并记录
for layer_index in range(27,28):  # 28 层
    layer_folder = os.path.join(glm_activation_folder, str(layer_index + 1))
    os.makedirs(layer_folder, exist_ok=True)
    if hasattr(model.transformer, 'layers') and layer_index < len(model.transformer.layers):
        target_glm_block = model.transformer.layers[layer_index]
    else:
        print("layer:", layer_index, "不存在")
        continue
    for neuron_index in range(0,4096):  # 每层 4096 个神经元
        num_samples_per_neuron = random.randint(10, 15)  # 随机选择样本数
        sampled_texts = df.sample(n=num_samples_per_neuron)["text"].tolist()  # 随机选择样本文本
        neuron_records_all = []  # 存储所有神经元的激活值记录
        for text in sampled_texts:
            neuron_records = []
            inputs = tokenizer(text, return_tensors="pt", padding=False, truncation=True).to(device)
            try:
                with torch.no_grad():
                    output = model(**inputs)
            except:
                continue
            # 注册hook以捕获激活值
            handle = target_glm_block.register_forward_hook(hook(inputs, output))
            handle.remove()
            neuron_records_all.append(neuron_records)
        neuron_record_file = os.path.join(layer_folder, f"{neuron_index}.json")
        neuron_records_dict_list = []
        for i in neuron_records_all:
            neuron_records_dict_list.append([asdict(record) for record in i])
        with open(neuron_record_file, 'w') as f:
            json.dump(neuron_records_dict_list, f, indent=4)